In [2]:
search_for = 132
start_from = 0
threads = 10
thread = 1
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.004218101501464844
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 375
all cases: 1268606955
type: [1, 1, 1, 1, 132] 132
cases of this type: 303595776
100000 73.99306346759585
200000 114.46394806127199
300000 115.86721243632263
400000 115.10190623543683
500000 116.31635595390092
600000 114.66476140512995
700000 118.7251744147205
800000 113.3532273697294
900000 107.9714906249547
1000000 101.02089225543939
1100000 93.81865411162106
1200000 26.180658785503297
1300000 95.46544622753598
1400000 93.516727250991
1500000 93.17614175755416
1600000 92.45302539468861
1700000 93.70494438334413
1800000 99.20103366870825
1900000 93.37690720325868
2000000 94.16547706770021
2100000 94.1157054621333
2200000 95.54230765792768
2300000 74.4777367877982
2400000 45.8656044131364
2500000 92.97357987911502
2600000 91.94889888929752
2700000 91.99124191458476
2800000 92.26253068596331
2900000 94.52283430622457
3000000 93.330212283251

29600000 97.25549292748371
29700000 68.85154829283813
29800000 75.86411072200256
29900000 102.93184033580775
30000000 101.64331373752997
30100000 102.1098135603223
30200000 42.55552040568096
30300000 101.73925432526694
30400000 104.20202044492433
30500000 101.75644093182976
30600000 98.98764073517474
30700000 100.93680618613313
30800000 98.92037202845418
30900000 98.02799609894889
31000000 92.69049723522949
31100000 26.607319121108073
31200000 98.80581544016746
31300000 96.73988089076957
31400000 97.16640040003342
31500000 100.6232593984107
31600000 99.58037399972409
31700000 99.78515683477939
31800000 102.15842235507408
31900000 101.48239209926561
32000000 57.00303353990365
32100000 89.17959740463287
32200000 105.93659580079341
32300000 99.44968515039524
32400000 101.04655340029274
32500000 34.31554028791625
32600000 101.7980985487059
32700000 100.68855068217465
32800000 98.02442735630962
32900000 100.6164738207311
33000000 100.04502469335569
33100000 94.50711268516734
33200000 93.142

59900000 96.32250574607077
60000000 95.55247168429861
60100000 98.86128882038817
60200000 97.47018435536427
60300000 36.86125326029701
60400000 94.26213793797989
60500000 90.48269833411125
60600000 90.92469114562859
60700000 88.03130117234235
60800000 85.77410550994541
60900000 85.7508944389968
61000000 23.56658383038945
61100000 91.76536296417288
61200000 86.88988248735085
61300000 87.02713200462972
61400000 86.55706978867616
61500000 90.34459951024907
61600000 93.17003451602685
61700000 36.80438206861745
61800000 100.43954661334922
61900000 99.42346993957993
62000000 97.0217537695895
62100000 101.49248219535714
62200000 97.76751076444587
62300000 98.23703640490804
62400000 99.88561280203369
62500000 97.61486141987844
62600000 50.82439148687144
62700000 83.31578742491998
62800000 93.72947488579815
62900000 90.18819289198862
63000000 85.95283657514142
63100000 87.12091779140646
63200000 89.37008788288442
63300000 23.55423626827025
63400000 93.5898146936333
63500000 89.7471356245921
636

90400000 52.36519286199728
90500000 63.44758821802236
90600000 81.88131648679305
90700000 78.65855170655159
90800000 80.8162532138874
90900000 20.57234980446474
91000000 78.97777320221839
91100000 72.96747422667036
91200000 78.91515047578052
91300000 70.03103256912195
91400000 40.605503163606684
91500000 80.82510711597989
91600000 79.87986269603351
91700000 86.33202046267128
91800000 84.5622749247961
91900000 84.60423198408829
92000000 87.47840664543156
92100000 84.74498407538583
92200000 86.8787349102736
92300000 83.82006165367726
92400000 79.95177024810332
92500000 81.6396657235689
92600000 78.0566095263554
92700000 58.27915014372953
92800000 51.16046549145622
92900000 71.0126672420815
93000000 79.10027946822673
93100000 58.64256317625582
93200000 17.056141689127994
93300000 79.70057029321592
93400000 66.48244136616145
93500000 68.64726393912605
93600000 56.902856692903946
93700000 54.03270196077924
93800000 74.58495662769444
93900000 76.38061505647256
94000000 67.21967048685431
9410

119900000 63.916320635682716
120000000 61.480933225663335
120100000 60.294652265784194
120200000 56.72487582996138
120300000 41.0999360894831
120400000 59.60303564920593
120500000 50.28666911289906
120600000 25.956916361240932
120700000 40.56572555738669
120800000 14.487754062724672
120900000 44.864676190424234
121000000 28.04632961502984
121100000 32.28034434993124
121200000 58.848774883809696
121300000 38.18532181249077
121400000 42.29508093680731
121500000 49.46417124711782
121600000 63.654237111207294
121700000 48.942633122426194
121800000 61.59002650569576
121900000 67.2712142531974
122000000 73.48294561870142
122100000 46.76163474726685
122200000 47.70305313943251
122300000 55.11420451522818
122400000 41.86529306709923
122500000 46.83258368353622
122600000 51.88086332892417
122700000 39.20553377249491
122800000 42.415799532586526
122900000 19.266875280257395
123000000 48.42318400536701
123100000 11.865505993534462
123200000 45.95806415114337
123300000 24.259555873402775
123400000

...

1190500000 3.3654321564287013
1190600000 2.510086302463104
1190700000 3.011293432515234
1190800000 3.047237209130257
1190900000 2.6645219124932193
1191000000 3.0855539135017693
1191100000 3.2447256857473223
1191200000 3.4111753873134085
1191300000 2.986550559038351
1191400000 3.4830962607128453
1191500000 3.055350960655408
1191600000 4.0629178284385405
best so far: 0
type: [1, 4, 1, 3, 11] 132
cases of this type: 1581228
1191700000 2.372307667511642
1191800000 1.561796035502318
1191900000 1.1794001860237717
1192000000 1.5744790103385613
1192100000 1.6799368904063239
1192200000 1.2218934741681218
1192300000 1.606470485147645
1192400000 2.159816737516022
1192500000 1.5888461160030463
1192600000 1.3159602768352794
1192700000 2.081578907801194
1192800000 1.4926684908958376
1192900000 1.24955314417545
1193000000 1.5967536865424117
1193100000 1.6422564527609744
1193200000 1.2186115530743102
best so far: 0
type: [1, 4, 1, 11, 3] 132
cases of this type: 431244
1193300000 1.8562120201208856
119

1204300000 2.779165473530498
1204400000 2.326615429281079
1204500000 3.751846550748199
1204600000 2.7927853317127727
1204700000 2.3410794918229216
1204800000 3.3114467568238575
1204900000 2.4205165598898035
1205000000 2.7052471984736792
1205100000 3.6913403209573925
1205200000 2.4120862881360385
1205300000 3.014939769064403
1205400000 3.436742810247332
1205500000 2.3459045733221435
1205600000 2.8114009598132568
1205700000 3.182766309697648
1205800000 1.9330997116224269
1205900000 3.183136576373018
1206000000 3.2883417325866824
1206100000 2.0045081107856366
1206200000 3.090035429020951
1206300000 3.1963388162728252
1206400000 1.9261301998495028
1206500000 3.1855668330891764
1206600000 2.9413616448753577
1206700000 2.096777802174207
1206800000 3.122883201875806
1206900000 3.063250934637487
1207000000 2.533392599803309
1207100000 1.9047215870290035
1207200000 0.6035317474133373
1207300000 0.4819170605829557
1207400000 0.5721858275326756
1207500000 0.5898363412711124
1207600000 0.579715416

1231400000 0.6033826741459237
1231500000 0.9732038639475702
1231600000 0.9283167775076363
1231700000 0.6256398350906438
1231800000 1.0068185200710793
1231900000 0.931800010789891
1232000000 0.6892382444889049
1232100000 0.9964078702045281
1232200000 0.8118177714200318
1232300000 0.7313580797409284
1232400000 0.9566017258987725
1232500000 0.8578982451086011
1232600000 0.8077254501864215
1232700000 0.8901620549484789
1232800000 0.6654895617820819
1232900000 0.8854765070517494
best so far: 0
type: [2, 1, 1, 6, 11] 132
cases of this type: 3162456
1233000000 0.9664386502418321
1233100000 0.6062272492554122
1233200000 0.8393765169834495
1233300000 0.9478890567078888
1233400000 0.6116282406032085
1233500000 0.9453754174179434
1233600000 0.9396645254154703
1233700000 0.5126236255635686
1233800000 0.8523768518182934
1233900000 0.7967922456110617
1234000000 0.5178747605987853
1234100000 1.0128441218709283
1234200000 0.6890337609156668
1234300000 0.5551788470731676
1234400000 0.8128454511899816
1

1252800000 0.3492864230742256
1252900000 0.24772910013581978
1253000000 0.24554722523714106
1253100000 0.3334634319205582
1253200000 0.2476238214339647
best so far: 0
type: [2, 2, 1, 3, 11] 132
cases of this type: 790614
1253300000 0.26331070352473523
1253400000 0.17327861073773104
1253500000 0.1514541359285315
1253600000 0.22158003132491974
1253700000 0.2063195307581524
1253800000 0.20074611091346542
1253900000 0.2054822385621104
1254000000 0.17353702609953284
best so far: 0
type: [2, 2, 1, 11, 3] 132
cases of this type: 215622
1254100000 0.13916039652365114
1254200000 0.11479540805604722
best so far: 0
type: [2, 2, 1, 33, 1] 132
cases of this type: 71874
1254300000 0.09449237757478157
best so far: 0
type: [2, 2, 3, 1, 11] 132
cases of this type: 263538
1254400000 0.21121908738789957
1254500000 0.10468209069173534
best so far: 0
type: [2, 2, 3, 11, 1] 132
cases of this type: 23958
best so far: 0
type: [2, 2, 11, 1, 3] 132
cases of this type: 19602
1254600000 0.17693980429847042
best s

best so far: 0
type: [3, 4, 1, 11, 1] 132
cases of this type: 5324
best so far: 0
type: [3, 4, 11, 1, 1] 132
cases of this type: 484
best so far: 0
type: [3, 11, 1, 1, 4] 132
cases of this type: 2816
best so far: 0
type: [3, 11, 1, 2, 2] 132
cases of this type: 1408
best so far: 0
type: [3, 11, 1, 4, 1] 132
cases of this type: 704
best so far: 0
type: [3, 11, 2, 1, 2] 132
cases of this type: 704
best so far: 0
type: [3, 11, 2, 2, 1] 132
cases of this type: 352
best so far: 0
type: [3, 11, 4, 1, 1] 132
cases of this type: 176
best so far: 0
type: [3, 22, 1, 1, 2] 132
cases of this type: 352
best so far: 0
type: [3, 22, 1, 2, 1] 132
cases of this type: 176
best so far: 0
type: [3, 22, 2, 1, 1] 132
cases of this type: 88
best so far: 0
type: [3, 44, 1, 1, 1] 132
cases of this type: 44
best so far: 0
type: [4, 1, 1, 1, 33] 132
cases of this type: 1185921
1266100000 0.023805327629937065
1266200000 0.06577667118203309
1266300000 0.04127573271150391
1266400000 0.03997282899064819
1266500000 0